In [379]:
import numpy as np
import math



def MLSSVRTrain(trnX, trnY, gamma, lambdaa, p):
    if (len(trnX) != len(trnY)):
        print('The number of rows in trnX and trnY must be equal.')
        return 0, 0
    
    l = np.shape(trnY)[0]
    m = np.shape(trnY)[1]
    if not gamma or  not lambdaa:
        gamma=1
        lambdaa = 1
    K = Kerfun('rbf', trnX, trnX, p, 0)
    H = np.tile(K, (m, m))+ np.identity(m * l) / gamma

    P = np.zeros((m*l,m))
    print(K)
    print(H)
    for  t in range(1,m+1):
        idx1 = l * (t - 1) + 1;
        idx2 = l * t;
#dangerous index
        # for i in xrange(idx1,idx2):
        #     for j in xrange(idx1,idx2):
        #         H(i,j) = H(i,j)+ K*(m/lambda)
        #     P(i , t) = np.ones((l,1))
        H[idx1-1:idx2, idx1-1:idx2] = H[idx1-1:idx2, idx1-1:idx2] + K*(m/lambdaa) 
        P[idx1-1:idx2 , t-1] = np.transpose(np.ones((l,1)))


    eta, _, _, _ = np.linalg.lstsq(H,P)
    ttrnY = np.transpose(trnY.ravel())
    nu, _, _, _ = np.linalg.lstsq(H,ttrnY)
    S = np.dot(np.transpose(P),eta)
    b = np.dot(np.dot(np.linalg.inv(S), np.transpose(eta)), ttrnY)
    alpha = nu - np.dot(eta, b)
    alpha = np.reshape(alpha,(l,m))

    return alpha, b


In [419]:
def Kerfun(kernel,X,Z,p1,p2):
	sx  = np.shape(X)
	sz  = np.shape(Z)
	if sx[1] != sz[1]:
		K = []
		print("The second dimensions for X and Z must be agree.")
		return K

	if kernel.lower == 'linear':
		K = np.dot(X,np.transpose(Z))
	elif kernel.lower == 'poly':
		K = np.power((np.dot(X,np.transpose(Z))+p1),p2)
	elif kernel.lower == 'rbf':
		K = np.exp(-p1*np.tile(np.sum(train_instance*train_instance, axis=1),(sz[0],1))+np.tile(np.transpose(np.sum(Z*Z, axis=1)),(sx[0],1))-2*X.dot(np.transpose(Z)))	elif kernel.lower == 'erbf':
		#K = np.exp(-p1*np.tile(np.sum(X*X, axis=1),(1,sz[0]))+np.transpose(np.sum(Z*Z, axis=1)).reshape(sx[0],1)-2*X.dot(np.transpose(Z)))/(2*np.power(p1,2))+p2
	elif kernel.lower == 'sigmoid':
		K = np.tanh(p1*X.dot(np.transpose(Z))/sx[1]+p2)
	else:
		K = X.dot(np.transpose(Z))+p1+p2
	return K



SyntaxError: invalid syntax (<ipython-input-419-38683b6bc0fa>, line 14)

In [298]:
import numpy as np
import math




def MLSSVRPredict(tstX, tstY, trnX, alpha, b, lambdaa, p):
    if np.shape(tstY)[1] != len(b):
        print('The number of column in tstY and b must be equal.')
        return

    #m = len(tstY.columns);
    m = np.shape(tstY)[1]
    #l = len(trnX);
    l = np.shape(trnX)[0]
    
    if (np.shape(alpha)[0] != l or np.shape(alpha)[1] != m):
        print('The size of alpha should be ' + l + '*' + m)
        return

    tstN = np.shape(tstX)[0];
    #b = np.reshape(b,[],1)
    b = np.transpose(b.ravel())

    K = Kerfun('rbf', tstX, trnX, p, 0)
    #predictY = repmat( sum(K*alpha, 2), 1, m) + K*alpha*(m/lambda) + repmat(b', tstN, 1);

    #predictY = np.tile(np.sum(np.dot(K,alpha), axis=1), (l, m)) + np.dot(K,alpha)*(m/lambdaa) + np.tile(np.transpose(b), (tstN, l))
    predictY = np.transpose(np.tile(np.transpose(np.sum(np.dot(K,alpha),axis=1)), (m,1 ))) + np.dot(K,alpha)*(m/lambdaa) + np.tile(np.transpose(b), (tstN, 1))


    TSE = np.zeros((1,m))
    R2 = np.zeros((1,m))
    for t in range(m):
        ppp = predictY[:, t]
        ttt = tstY[:, t]
   
        TSE[0,t] = np.sum((ppp-ttt)**2)
        R = np.corrcoef(ppp, tstY[:,t])
        if (len(R)>1):
            R2[0,t] = R[0,1]**2
    return predictY, TSE, R2


In [420]:
import random
import numpy as np
import math
def GridMLSSVR(trnX,trnY,fold):

 	gamma = np.power(np.arange(-5, 15, 2),2)
 	lambdaa = np.power(np.arange(-10, 10, 2),2)
 	p = np.power(np.arange(-15, 3, 2),2)
 	tYs = np.shape(trnY)
 	m = tYs[1]

 	trnX,trnY = random_perm(trnX, trnY);

 	MSE_best =  math.inf

 	MSE = np.zeros((fold, m))
 	curR2 = np.zeros((1, m))
 	R2 = np.zeros((1, m))

 	sp = np.shape(p)
 	sg = np.shape(gamma)
 	sl = np.shape(lambdaa)
 	for i in range(sg[0]):
 		for j in range(sl[0]):
 			for k in range(sp[0]):
 				predictY = []
 				for v in range(fold):
 					train_inst, train_lbl, test_inst, test_lbl = folding(trnX, trnY, fold, v)
 					print(train_inst)
 					print(train_lbl)
 					print(test_inst)
 					print(test_lbl)
 					return
 					
 					alpha, b = MLSSVRTrain(train_inst, train_lbl, gamma[i], lambdaa[j], p[k])
 					tmpY, ms, xx = MLSSVRPredict(test_inst, test_lbl,train_inst,alpha,b,lambdaa[j],p[k])
 					tmpY, ms, xx = MLSSVRPredict(test_inst, test_lbl,train_inst,alpha,b,lambdaa[j],p[k])
 					tmpY, ms, xx = MLSSVRPredict(test_inst, test_lbl,train_inst,alpha,b,lambdaa[j],p[k])
 					MSE[v,:] = ms

 					if not np.shape(predictY)[0]:
 						predictY = tmpY
 					else:	
 						predictY = np.concatenate((predictY, tmpY), axis=0)

 				sy = np.shape(trnY)		
 				curMSE = np.sum(MSE)/ (sy[0]*sy[1]); 		

 				if MSE_best > curMSE:
 				    gamma_best = gamma[i]
 				    lambda_best = lambdaa[j]
 				    p_best = p[k]

 	#print('gamma = ' + str(math.log2(gamma[i]) + ' lambda = ' +str(math.log2(lambdaa[j])) + ' p = ' +str(math.log2(p[k])) +' mean_MSE = ' + str(curMSE) +' (best :' +str(math.log2(gamma_best))+' '+ str(math.log2(lambda_best))+' ' +str(math.log2(p_best))+' '+str(MSE_best)))
 	print(math.log2(gamma_best))
 	print(math.log2(lambda_best))
 	print(math.log2(p_best))
 	print(MSE_best)


def random_perm(svm_inst, svm_lbl):
    
    random.seed(a=1, version=2)
    n = np.shape(svm_inst)[0]
    for i in range(n):
        k = round(i + (n - i) * random.random())-1
        svm_inst[[k, i], :] = svm_inst[[i, k], :]
        svm_lbl[[k, i], :] = svm_lbl[[i, k], :]
    return svm_inst, svm_lbl




def folding(svm_inst, svm_lbl, fold, k):
    n = np.shape(svm_inst)[0]
    start_index = round((k - 1)*n/fold) + 1;
    end_index = round(k*n/fold);
    test_index = [start_index, end_index]

    test_inst = svm_inst[test_index, :];
    test_lbl = svm_lbl[test_index, :];

    train_inst = svm_inst;
    #train_inst[test_index, :] = [];
    np.delete(train_inst, test_index, axis=0)
    train_lbl = svm_lbl;
    #train_lbl[test_index, :] = [];
    np.delete(train_lbl, test_index, axis=0)

    return train_inst, train_lbl, test_inst, test_lbl


In [283]:
import pandas as pd
df = pd.read_csv('test1.csv')

In [284]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import preprocessing, linear_model
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import ShuffleSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
def train_test_spliter(df, random_seed=0, test_size=0.20, n_splits=1):
    trainm = preprocessing.maxabs_scale(df, axis=0, copy=True)
    col = df.columns
    df1 = pd.DataFrame(data=trainm, columns=col)
    rs = ShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=random_seed)
    for train_index, test_index in rs.split(df):
        pass
    traindf = pd.DataFrame(data=df, index=train_index)
    testdf = pd.DataFrame(data=df, index=test_index)
    return traindf, testdf

In [285]:
traindf, testdf = train_test_spliter(df)

In [286]:
train_m = traindf.values

In [287]:
test_m = testdf.values

In [288]:
train_instance =train_m[:,0:3]

In [289]:
train_label = train_m[:,3:5]

In [290]:
test_instance =test_m[:,0:3]

In [291]:
test_label = test_m[:,3:5]

In [324]:
import scipy.io as sio
sio.savemat('trnX.mat',{'Matrix1':train_instance})
sio.savemat('trnY.mat',{'Matrix1':train_label})
sio.savemat('tstX.mat',{'Matrix1':test_instance})
sio.savemat('tstY.mat',{'Matrix1':test_label})

In [421]:
alpha,b = MLSSVRTrain(train_instance,train_label,2,2,8)

[[ 8.41832829  8.52816251  8.44793758 ...,  8.26948624  8.423839
   8.70830761]
 [ 8.52816251  9.59113502  8.77757487 ...,  9.06463991  8.60083841
   9.69232423]
 [ 8.44793758  8.77757487  8.87336361 ...,  8.28283474  8.46469128
   8.86601267]
 ..., 
 [ 8.26948624  9.06463991  8.28283474 ...,  8.82696161  8.32664475
   9.11934792]
 [ 8.423839    8.60083841  8.46469128 ...,  8.32664475  8.43414658
   8.77530364]
 [ 8.70830761  9.69232423  8.86601267 ...,  9.11934792  8.77530364
   9.90485181]]
[[  8.91832829   8.52816251   8.44793758 ...,   8.26948624   8.423839
    8.70830761]
 [  8.52816251  10.09113502   8.77757487 ...,   9.06463991   8.60083841
    9.69232423]
 [  8.44793758   8.77757487   9.37336361 ...,   8.28283474   8.46469128
    8.86601267]
 ..., 
 [  8.26948624   9.06463991   8.28283474 ...,   9.32696161   8.32664475
    9.11934792]
 [  8.423839     8.60083841   8.46469128 ...,   8.32664475   8.93414658
    8.77530364]
 [  8.70830761   9.69232423   8.86601267 ...,   9.1193479

In [417]:
K = np.exp(-1*np.tile(np.sum(train_instance*train_instance, axis=1),(sz[0],1))+np.tile(np.transpose(np.sum(Z*Z, axis=1)),(sx[0],1))-2*X.dot(np.transpose(Z)))

In [418]:
K

array([[ 0.34308394,  0.16358481,  0.32720321, ...,  0.27429407,
         0.54023836,  0.12777509],
       [ 0.19707541,  0.03959158,  0.22623614, ...,  0.12855765,
         0.25624468,  0.0334301 ],
       [ 0.73865924,  0.42393484,  0.31173698, ...,  0.52506304,
         0.52761082,  0.29558979],
       ..., 
       [ 0.24469953,  0.09519742,  0.20749211, ...,  0.13517075,
         0.32255682,  0.05615317],
       [ 0.63749287,  0.25098936,  0.27578906, ...,  0.42665754,
         0.39223895,  0.18690244],
       [ 0.16132083,  0.03503422,  0.16531304, ...,  0.07946975,
         0.19997206,  0.02215474]])

In [407]:
np.sum(train_instance*train_instance, axis=1)

array([ 0.41832829,  1.59113502,  0.87336361,  0.59713403,  0.53439622,
        1.77808048,  0.57040175,  0.35007646,  0.11512263,  1.26179476,
        1.93675289,  0.65145184,  1.18182838,  0.62016309,  0.52960715,
        0.29223186,  1.57511057,  0.93358906,  1.77451102,  1.71442468,
        1.44340127,  1.12615777,  1.21888039,  1.02300702,  1.07728249,
        1.26987946,  1.27909444,  0.70997937,  1.11704777,  0.34074413,
        0.91918982,  0.76973877,  1.03046662,  0.91480019,  0.4659325 ,
        1.99746778,  0.94278369,  1.44387651,  0.91841314,  0.75288704,
        1.02893282,  1.54031964,  0.13337209,  0.0737663 ,  1.37965691,
        0.46286275,  1.27707926,  0.89076766,  1.13659763,  0.93184971,
        1.80865233,  0.34200933,  1.83305501,  0.53210712,  1.63390743,
        0.86541036,  0.54775405,  1.11745547,  1.32753127,  1.15656292,
        1.03989819,  0.98526146,  1.56235696,  0.49286529,  0.39447875,
        0.6136574 ,  0.76100998,  1.29511163,  0.30248772,  1.48

In [415]:
sum(np.transpose(np.multiply(train_instance,train_instance)))

array([ 0.41832829,  1.59113502,  0.87336361,  0.59713403,  0.53439622,
        1.77808048,  0.57040175,  0.35007646,  0.11512263,  1.26179476,
        1.93675289,  0.65145184,  1.18182838,  0.62016309,  0.52960715,
        0.29223186,  1.57511057,  0.93358906,  1.77451102,  1.71442468,
        1.44340127,  1.12615777,  1.21888039,  1.02300702,  1.07728249,
        1.26987946,  1.27909444,  0.70997937,  1.11704777,  0.34074413,
        0.91918982,  0.76973877,  1.03046662,  0.91480019,  0.4659325 ,
        1.99746778,  0.94278369,  1.44387651,  0.91841314,  0.75288704,
        1.02893282,  1.54031964,  0.13337209,  0.0737663 ,  1.37965691,
        0.46286275,  1.27707926,  0.89076766,  1.13659763,  0.93184971,
        1.80865233,  0.34200933,  1.83305501,  0.53210712,  1.63390743,
        0.86541036,  0.54775405,  1.11745547,  1.32753127,  1.15656292,
        1.03989819,  0.98526146,  1.56235696,  0.49286529,  0.39447875,
        0.6136574 ,  0.76100998,  1.29511163,  0.30248772,  1.48

In [410]:
b= [ 0.87782816,  0.37283318,  0.34047368]
sum(b)

1.5911350199999998

In [416]:
train_instance

array([[  2.70521200e-02,   2.56793335e-01,   5.93003929e-01],
       [  9.36924841e-01,   6.10600672e-01,   5.83501225e-01],
       [  3.55437440e-02,   8.50630449e-01,   3.85393419e-01],
       [  4.30675907e-01,   3.86140628e-01,   5.12394093e-01],
       [  2.05115239e-01,   1.00419294e-01,   6.94434968e-01],
       [  9.47525056e-01,   3.92745060e-01,   8.52072803e-01],
       [  1.87506445e-01,   7.28086065e-01,   7.16502790e-02],
       [  3.76877263e-01,   3.61884966e-01,   2.77631517e-01],
       [  1.08017700e-03,   3.02158161e-01,   1.54343486e-01],
       [  4.96599403e-01,   1.38440515e-01,   9.98007024e-01],
       [  9.65217846e-01,   7.94080702e-01,   6.11999374e-01],
       [  7.01700852e-01,   1.40013388e-01,   3.73448800e-01],
       [  4.67573498e-01,   6.92256536e-01,   6.95689795e-01],
       [  2.03655064e-01,   1.86339761e-01,   7.37539964e-01],
       [  7.19767498e-01,   4.97040000e-04,   1.07432083e-01],
       [  8.14190920e-02,   1.13039538e-01,   5.2232638